In [ ]:
!df -h

In [1]:
!cat .env

STAGE=ENGIE_PRODUCTION


In [1]:
Company.objects.all()

<QuerySet [<Company: a17b7aca-5552-4f17-9ba0-9946029390a6: POC-Projeto APR>, <Company: 571581e3-da3a-469e-8676-0dc58c056ef7: GSO - UHE-SALTO OSORIO>, <Company: 4920816b-507a-48b1-86d5-d2eef6c63a1f: GSO - UTPS>, <Company: cff59760-69dd-4d07-bba5-61aa8b6faeea: GES - UHE-CANA BRAVA>, <Company: 90041917-e41e-46f2-b979-7d3ad53efad2: Engie Umburanas>, <Company: 69c4238a-fc42-4a16-a312-364ff4cd10c6: GSB - UHE-ITA>, <Company: 4b4bfdad-f8b8-403b-93a6-26e846a6770b: GSO - UHE-MACHADINHO>, <Company: 1b1511d1-6f28-4aed-b02c-7c820c5f9dbb: GSO - PCH-RONDONÓPOLIS>, <Company: e9fce210-f652-4d7d-befe-32e02a8f0f60: GSO - UHE-ITA>, <Company: 6ea3a49c-55a7-4aeb-919f-949034a5d2fa: GSB - UHE-ESTREITO>, <Company: e2d725b7-1b3b-4138-b790-cf49a6ab5284: GSB - UHE-JAGUARA>, <Company: 6af632a4-8b09-4bc2-bc6c-43d9a7dc23f4: GSB - UHE-MIRANDA>, <Company: 76ddb208-b049-4022-9736-9fd0101a5392: GSB - UHE-PONTE DE PEDRA>, <Company: 247c952a-9371-483d-8039-9a65b41b8db3: GSB - UHE-SÃO SALVADOR>, <Company: 24072e01-a4a7-477

In [2]:
from django.db.models import Count, Q
import pandas as pd

In [3]:
def generate_excel(lista_df,name=''):   
    
    writer = pd.ExcelWriter(name+'engieRel.xlsx', engine='xlsxwriter')
    for i,a in enumerate(lista_df):
    # Write each DataFrame to a different worksheet
        a.to_excel(writer, sheet_name=name+str(i) or str(i))
    writer.save()

In [4]:
lista_df=[]

### Registros (sociofamiliar, edificação de uso coletivo, edificação sem habilitação, etc.): unidade + número de registros para aquela unidade, para o que está dentro da ZAS e o que está na ZSS, separado

In [5]:
zas=OccurrenceRecord.objects.values('company__name','occurrence_type__name').annotate(total_zas=Count('place_on_dam', filter=Q(place_on_dam='ZAS'))).filter(company__name__startswith='GSO',total_zas__gte=1).order_by('company__name','occurrence_type__name')
zss=OccurrenceRecord.objects.values('company__name','occurrence_type__name').annotate(total_zss=Count('place_on_dam', filter=Q(place_on_dam='ZSS'))).filter(company__name__startswith='GSO',total_zss__gte=1).order_by('company__name','occurrence_type__name')
geral_s_zssEZas=OccurrenceRecord.objects.filter(company__name__startswith="GSO").exclude(Q(place_on_dam='ZAS') | Q(place_on_dam='ZSS')).values('company__name','occurrence_type__name').annotate(total_registro=Count('uuid')).order_by('company__name','occurrence_type__name')

In [6]:
dfZas=pd.DataFrame(data=zas)
dfZss=pd.DataFrame(data=zss)
dfGeral=pd.DataFrame(data=geral_s_zssEZas)

In [48]:
lista_df.append(dfZas)
lista_df.append(dfZss)
lista_df.append(dfGeral)

In [49]:
generate_excel(lista_df,'registros')
lista_df=[]

### Composição por Idade (faixa etária e número de pessoas por faixa)

In [7]:
faixa_etaria=list(OccurrenceRecord.objects.filter(company__name__startswith='GSO',
                                             form_data__has_key='age').values('company__name',
                                                                              'form_data__age',
                                                                              'form_data__people').order_by('company__name'))

In [8]:
for i,a in enumerate(faixa_etaria):
    if a['form_data__people'] is not None:
        for ib,b in enumerate(a['form_data__people']):
            if 'age' in b:
                faixa_etaria[i]['form_data__integranteAge_'+str(ib)]=b['age']

In [9]:
dfFaixa=pd.DataFrame(data=faixa_etaria)

In [10]:
dfFaixa

,company__name,form_data__age,form_data__people,form_data__integranteAge_0,form_data__integranteAge_1,form_data__integranteAge_2,form_data__integranteAge_3,form_data__integranteAge_4,form_data__integranteAge_5,form_data__integranteAge_6,form_data__integranteAge_7,form_data__integranteAge_8,form_data__integranteAge_9,form_data__integranteAge_10,form_data__integranteAge_11,form_data__integranteAge_12,form_data__integranteAge_13
0,GSO - UHE-CANA BRAVA,71.0,"[{'age': 80, 'stay': '1', 'full_name': 'Lázaro...",80.0,61.0,19.0,47.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GSO - UHE-CANA BRAVA,57.0,"[{'age': 54, 'stay': '1', 'full_name': 'Angeli...",54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GSO - UHE-CANA BRAVA,55.0,"[{'age': 46, 'stay': '1', 'full_name': 'Antôni...",46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GSO - UHE-CANA BRAVA,34.0,"[{'age': 33, 'stay': '1', 'full_name': 'Jefers...",33.0,14.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GSO - UHE-CANA BRAVA,37.0,"[{'age': 27, 'stay': '1', 'full_name': 'Valdir...",27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4113,GSO - UHE-SÃO SALVADOR,53.0,"[{'age': 59, 'stay': '1', 'full_name': 'Pedro ...",59.0,18.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4114,GSO - UHE-SÃO SALVADOR,80.0,"[{'age': 39, 'stay': '1', 'full_name': 'Danilo...",39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4115,GSO - UTPS,53.0,"[{'age': 60, 'stay': '1', 'full_name': 'Lei Ch...",60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4116,GSO - UTPS,49.0,"[{'age': 39, 'stay': '1', 'full_name': 'Matizz...",39.0,22.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
lista_df.append(dfFaixa)

In [93]:
generate_excel(lista_df,'faixa_etaria')
lista_df=[]

### Residentes Fixos e de Mobilidade Reduzida (com descrição do motivo, como gestação ou obesidade) 

In [108]:
fixos_mobi_reduz=OccurrenceRecord.objects.filter(Q(form_data__has_key='people') | Q(form_data__has_key='conditions'),company__name__startswith='GSO').values('company__name',
                                                                              'form_data__people','form_data__conditions','form_data__main_contact_name').order_by('company__name')

In [109]:
#SocioFamiliar
occ = OccurrenceType.objects.get(uuid='f36530bb-d8f9-4910-be86-7d4ca324539d')

In [110]:
def traduzirInner(value,occ,campo):
    for a in occ.form_fields['fields']:
        if 'people' == a['apiName']:
            for c in a['innerFields']:
                if c['apiName'] == campo:
                    for b in c['selectOptions']['options']:

                        if b['value'] == value:
                            return b['name']

In [111]:
def traduzirSelect(value,occ,campo):
    for a in occ.form_fields['fields']:
        if a['apiName'] == campo:
            for b in a['selectOptions']['options']:
                if b['value'] == value:
                    return b['name']

In [114]:
for i,a in enumerate(fixos_mobi_reduz):
    if a['form_data__people'] is not None:
        size=len(a['form_data__people']) + 1 if a['form_data__main_contact_name'] is not None else 0
        a['total_people'] = size
        for b in a['form_data__people']:
            fixos_mobi_reduz[i]['stay']= traduzirInner(b['stay'],occ,'stay') if 'stay' in b else ''
            fixos_mobi_reduz[i]['ocupation']= traduzirInner(b['ocupation'],occ,'ocupation') if 'ocupation' in b else ''
            fixos_mobi_reduz[i]['family_tree']= traduzirInner(b['family_tree'],occ,'familyTree') if 'family_tree' in b else ''
            fixos_mobi_reduz[i]['school_level']= traduzirInner(b['school_level'],occ,'schoolLevel') if 'school_level' in b else ''
    if a['form_data__conditions'] is not None:
        a['form_data__conditions'] = traduzirSelect(a['form_data__conditions'],occ,'conditions')
    

In [115]:
list(fixos_mobi_reduz)

[{'company__name': 'GSO - UHE-CANA BRAVA',
  'form_data__people': [{'age': 85,
    'stay': '1',
    'full_name': 'Jovencio Fernando de Almeida ',
    'ocupation': '7',
    'family_tree': '12',
    'school_level': '1'}],
  'form_data__conditions': 'Proprietário',
  'form_data__main_contact_name': 'Dionísia Moreira Ramos',
  'total_people': 2,
  'stay': 'Permanente',
  'ocupation': 'Aposentado',
  'family_tree': 'Esposo(a)',
  'school_level': 'Analfabeto'},
 {'company__name': 'GSO - UHE-CANA BRAVA',
  'form_data__people': [{'age': 33,
    'stay': '1',
    'full_name': 'Jeferson Magno Mendonça da Costa Caetano ',
    'ocupation': '9',
    'family_tree': '12',
    'school_level': '3',
    'contact_phone': '62-994891371',
    'other_ocupation': 'Gerente da propriedade '},
   {'age': 14,
    'stay': '1',
    'full_name': 'Jeferson Magno Mendonça da Costa Caetano Junior',
    'ocupation': '6',
    'family_tree': '2',
    'school_level': '3'},
   {'age': 11,
    'stay': '1',
    'full_name': '